In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from utils import *
from operator import itemgetter

/tmp/ipykernel_57256/2063284992.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-03 20:22:44.669260: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-03 20:22:44.716775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 20:22:44.716814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin

In [2]:
top10_df = pd.read_csv("./data/content_top10_df.csv")

top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.106061,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.021277,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.118919,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.867647,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.238255,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.778523,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.066434,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,3.960993,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.871212,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.160305,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


In [3]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3 # Removes user id, rating couting and avg rating
num_item_features = item_train.shape[1] - 1 # Remove movie id
uvs = 3 # user genre vector start
ivs = 3 # item genre vector start
u_s = 3 # user start of features columns to use in training
i_s = 1 # items start of features columns to use in training

In [4]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


In [5]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


In [6]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  3.5 4.  4.  4.5]


In [7]:
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

In [8]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test = train_test_split(y_train, train_size = 0.80, shuffle=True, random_state=1)

In [9]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
1,0,-1.0,-0.8,-0.7,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,1,-0.7,-0.5,-0.7,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
-1,-1,-0.2,0.3,-0.4,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,-1,0.6,0.5,0.5,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
-1,0,0.7,0.6,0.5,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


In [10]:
num_outputs = 32

user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
])

input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   147232    ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   148256    ['input_2[0][0]']             
                                                                                              

In [11]:
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss=cost_fn)

In [12]:
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=100)

Epoch 1/100
1273/1273 [==============================] - 11s 8ms/step - loss: 0.1224
Epoch 2/100
1273/1273 [==============================] - 9s 7ms/step - loss: 0.1115
Epoch 3/100
1273/1273 [==============================] - 8s 6ms/step - loss: 0.1070
Epoch 4/100
1273/1273 [==============================] - 9s 7ms/step - loss: 0.1033
Epoch 5/100
1273/1273 [==============================] - 8s 6ms/step - loss: 0.1000
Epoch 6/100
1273/1273 [==============================] - 8s 6ms/step - loss: 0.0975
Epoch 7/100
1273/1273 [==============================] - 8s 7ms/step - loss: 0.0949
Epoch 8/100
1273/1273 [==============================] - 9s 7ms/step - loss: 0.0925
Epoch 9/100
1273/1273 [==============================] - 7s 6ms/step - loss: 0.0899
Epoch 10/100
1273/1273 [==============================] - 7s 5ms/step - loss: 0.0878
Epoch 11/100
1273/1273 [==============================] - 7s 6ms/step - loss: 0.0858
Epoch 12/100
1273/1273 [==============================] - 9s 7ms/step - l

In [13]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 1s 2ms/step - loss: 0.0747


0.07465222477912903

In [14]:
def sq_distance(a, b):
    return np.sum((a - b) ** 2)

In [15]:
def compute_distance_matrix(vms):
    m = vms.shape[0]
    M = np.zeros((m, m))

    for i in range(m):
        for j in range(m):
            M[i][j] = sq_distance(vms[i, :], vms[j, :])

    return M

In [16]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))
vm_m = item_NN(input_item_m)
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)
model_m = tf.keras.Model(input_item_m, vm_m)
model_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                148256    
                                                                 
 tf.math.l2_normalize_2 (TF  (None, 32)                0         
 OpLambda)                                                       
                                                                 
Total params: 148256 (579.12 KB)
Trainable params: 148256 (579.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])

27/27 [==============================] - 0s 2ms/step


In [18]:
distance_matrix = compute_distance_matrix(vms)

In [19]:
m = distance_matrix.shape[0]
similar_movies_list = []

distance_matrix = ma.masked_array(distance_matrix, mask=np.identity(distance_matrix.shape[0]))

for i in range(m):
    movie = dict()
    min_idx = np.argmin(distance_matrix[i, i:])
    movie1_id = int(item_vecs[i, 0])
    movie2_id = int(item_vecs[min_idx, 0])
    distance = distance_matrix[i][min_idx]
    movie["movie1_id"] = movie1_id
    movie["movie2_id"] = movie2_id
    movie["distance"] = distance 
    
    similar_movies_list.append(movie)

In [20]:
top_10_similar_movies = sorted(similar_movies_list, key=itemgetter('distance'))[0:9]

In [21]:
disp = [["movie1", "genres", "movie2", "genres"]]

for i in range(len(top_10_similar_movies)):
    movie1_id = top_10_similar_movies[i]["movie1_id"]
    movie2_id = top_10_similar_movies[i]["movie2_id"]
    disp.append([
        movie_dict[movie1_id]["title"], movie_dict[movie1_id]["genres"],
        movie_dict[movie2_id]["title"], movie_dict[movie2_id]["genres"]
    ])

table = tabulate.tabulate(disp, tablefmt="html", headers="firstrow")
table

movie1,genres,movie2,genres
Lost in Translation (2003),Comedy|Drama|Romance,Adaptation (2002),Comedy|Drama|Romance
Save the Last Dance (2001),Drama|Romance,Mona Lisa Smile (2003),Drama|Romance
"Assassination of Jesse James by the Coward Robert Ford, The (2007)",Crime|Drama,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller
Knocked Up (2007),Comedy|Drama|Romance,Punch-Drunk Love (2002),Comedy|Drama|Romance
Slumdog Millionaire (2008),Crime|Drama|Romance,"Dreamers, The (2003)",Drama
Saved! (2004),Comedy|Drama,Punch-Drunk Love (2002),Comedy|Drama|Romance
"Sweetest Thing, The (2002)",Comedy|Romance,Crocodile Dundee in Los Angeles (2001),Comedy|Drama
Clash of the Titans (2010),Action|Adventure|Drama|Fantasy,"One, The (2001)",Action|Sci-Fi|Thriller
Stuck on You (2003),Comedy,"Sweetest Thing, The (2002)",Comedy|Romance


In [22]:
top_50_most_rated_movies = load_metadata()

top_50_most_rated_movies_recommendations = []

disp = ['movie_1_id', 'movie_2_id', 'recommendation_distance', 'movie_1', 'movie_1_genres', 'movie_2', 'movie_2_genres']

for i in range(len(top_50_most_rated_movies)):
    movie_id = top_50_most_rated_movies[i][0]
    recommendation = [similar_movies for similar_movies in similar_movies_list if similar_movies["movie1_id"] == movie_id][0]
    recommended_movie_id = recommendation['movie2_id']
    recommendation_distance = recommendation['distance']
    movie_title = movie_dict[movie_id]['title']
    recommended_movie_title = movie_dict[recommended_movie_id]['title']
    movie_genres = movie_dict[movie_id]['genres']
    recommended_movie_genres = movie_dict[recommended_movie_id]['genres']
    top_50_most_rated_movies_recommendations.append([
        movie_id, recommended_movie_id, recommendation_distance,
        movie_title, movie_genres,
        recommended_movie_title, recommended_movie_genres
    ])

table = tabulate.tabulate([disp, *top_50_most_rated_movies_recommendations], tablefmt='html', headers='firstrow')
table

movie_1_id,movie_2_id,recommendation_distance,movie_1,movie_1_genres,movie_2,movie_2_genres
4993,5015,1.54455,"Lord of the Rings: The Fellowship of the Ring, The (2001)",Adventure|Fantasy,Monster's Ball (2001),Drama|Romance
5952,5072,0.736951,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,Metropolis (2001),Animation|Sci-Fi
7153,8640,2.10986,"Lord of the Rings: The Return of the King, The (2003)",Action|Adventure|Drama|Fantasy,King Arthur (2004),Action|Adventure|Drama
4306,7323,1.38881,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Romance,"Good bye, Lenin! (2003)",Comedy|Drama
58559,6377,0.427748,"Dark Knight, The (2008)",Action|Crime|Drama,Finding Nemo (2003),Adventure|Animation|Children|Comedy
6539,48774,1.03167,Pirates of the Caribbean: The Curse of the Black Pearl (2003),Action|Adventure|Comedy|Fantasy,Children of Men (2006),Action|Adventure|Drama|Sci-Fi|Thriller
79132,5013,1.14662,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller,Gosford Park (2001),Comedy|Drama|Mystery
6377,44974,1.6041,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Hard Candy (2005),Drama|Thriller
4886,57368,1.88042,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Cloverfield (2008),Action|Mystery|Sci-Fi|Thriller
6874,4701,1.60815,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller,Rush Hour 2 (2001),Action|Comedy
